<a href="https://colab.research.google.com/github/ShoOstiK/0_git_basics/blob/master/04_database_adv_Adov_Dmitriy_PM21_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Работа с базами данных (2)

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа с базами данных"
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.sqlitetutorial.net/sqlite-index/
* https://docs.python.org/3/library/sqlite3.html#sqlite3.IntegrityError
* https://www.sqlitetutorial.net/sqlite-alter-table/
* https://www.sqlitetutorial.net/sqlite-create-view/
* https://habr.com/ru/post/664000/
* https://learnsql.com/blog/what-is-common-table-expression/


## Задачи для совместного разбора

In [ ]:
import pandas as pd
import sqlite3

# данные
students = pd.DataFrame(
    [
        ("Сотников Евгений Янович", 1),
        ("Степанова Виктория Константиновна", 1),
        ("Горелова Вероника Яновна", 2),
        ("Гришин Иван Романович", 3),
    ],
    columns=["name", "group_id"],
)
groups = list(zip([1, 2, 3], ["ПМ20-1", "ПМ20-2", "ПМ20-3"]))

con = sqlite3.connect("demo.sqlite")
con.execute("PRAGMA foreign_keys = 1")
cur = con.cursor()

# создаем таблицы
sql = """
DROP TABLE IF EXISTS StudentGroup;
DROP TABLE IF EXISTS Student;
CREATE TABLE StudentGroup (
    id int PRIMARY KEY,
    name varchar
);

CREATE TABLE Student(
    name VARCHAR PRIMARY KEY,
    group_id INT,
    FOREIGN KEY (group_id) REFERENCES StudentGroup(id)
);
"""
cur.executescript(sql)
con.commit()

# добавляем записи
sql = """
INSERT INTO StudentGroup(id, name) VALUES (?, ?)
"""
cur.executemany(sql, groups)
con.commit()

students.to_sql("Student", con, if_exists="append", index=False)

4

1\. Добавить столбец Age со значением по умолчанию. Добавить запись к таблицу

2\. Занумеруйте студентов в рамках каждой группы.

3\. Выведите уникальные номера студентов

## Лабораторная работа 4

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy` и `pandas`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy` или структур `pandas` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

__Для начала работы подключитесь к БД `recipes.db` и создайте объект-курсор.__

In [ ]:
con = sqlite3.connect("recipes.db")

cur = con.cursor()

<p class="task" id="1"></p>

1\. Создайте уникальный индекс для таблицы `Review` для обеспечения уникальности сочетания значений в полях `user_id` и `recipe_id`. 

In [ ]:
cur.execute('''CREATE UNIQUE INDEX unique_index

               ON Review (user_id, recipe_id);''')

In [ ]:
con.commit()

<p class="task" id="2"></p>

2\. Напишите функцию `add_review(review_id, user_id, recipe_id, date, rating, review)`, которая добавляет запись в таблицу `Review`. В случае успешного добавления функция должна вернуть значение 0. В случае нарушения ограничения целостности функция должна вернуть значение 1. В случае любых других ошибок функция должна вернуть значение 2. Продемонстрируйте работу функции, попытавшись добавить одну и ту же запись дважды в двух ячейках подряд.

Для решения задачи воспользуйтесь механизмом try - except и обработайте соответствующее исключение.

In [ ]:
def add_review(review_id, user_id, recipe_id, date, rating, review):

    try:

        cur.execute('INSERT INTO Review VALUES (?, ?, ?, ?, ?, ?);', (review_id, user_id, recipe_id, date, rating, review))

        return 0

    except sqlite3.IntegrityError:

        return 1

    except:

        return 2

In [ ]:
add_review(180, 405, 651, '11:22:1003', 3 ,'WLF')

0

In [ ]:
add_review(180, 405, 651, '11:22:1003', 3 ,'WLF')

1

<p class="task" id="3"></p>

3\. _Измените_ таблицу Review, добавив в нее поле `toxic` булева типа. 

<p class="task" id="4"></p>

4\. Вам дан классификатор `clf`, который классифицирует тексты отзывов как токсичные (`True`) и не токсичные (`False`).
Напишите функцию `classify_reviews`, которая итеративно получает пакет (батч) `batch_size` строк из таблицы Reviews, у которых не проставлено значение в столбце `toxic`, делает для них прогноз при помощи модели `clf` и обновляет соответствующие строки в БД. Данная процедура выполняется до тех пор, пока в БД есть строки, для которых требуется получить прогноз.

Продемонстрируйте результат, выведя на экран количество токсичных и не токсичных отзывов в таблице.

In [ ]:
from sklearn.dummy import DummyClassifier

clf = DummyClassifier(strategy="uniform").fit(None, [True, False])

<p class="task" id="5"></p>

5\. Создайте представление `RecipeWithYear`, в котором добавлен дополнительный столбец `year`, содержащий год даты из столбца `submitted`. Сделайте выборку из этого представления и выведите на экран количество рецептов с разбивкой по годам.

In [ ]:
view = cur.execute(''' CREATE VIEW RecipeWithYea AS
                       SELECT YEAR(submitted)
                       FROM Recipe
                ''')

In [ ]:
req = cur.execute('SELECT * FROM Recipe')

In [ ]:
req.description

(('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('minutes', None, None, None, None, None, None),
 ('submitted', None, None, None, None, None, None),
 ('description', None, None, None, None, None, None),
 ('n_ingredients', None, None, None, None, None, None))

In [ ]:
for row in req:
  print(row)

Выходные данные были обрезаны до нескольких последних строк (5000).
(141362, 'bean  potato and sauerkraut soup', 150, '2005-10-14', "raining 6 days straight here in the northeast and suddenly i'm thinking about warm comforting soups as a way to warm from the inside out.  after pulling out one of my soup cookbooks and came across this one and decided to post it for the zaar world tour.  you can make this vegetarian by omitting the prosciutto. i have not tried this one so please feel free to share your comments and sugeestions.", 13)
(66110, 'bean oat muffins', 35, '2003-07-04', "dont let the name throw you off. these are really sweet, moist muffins. a perfect breakfast snack! you dont taste beans in your muffins, if you didn't know you added the beans, you would never guess! i got this recipe from ", None)
(219761, 'beanie weenies', 9, '2007-03-30', 'my version of beans and franks.  one daughter likes the beans part and the other daughter likes the franks part.  even though this seems l

KeyboardInterrupt: ignored

<p class="task" id="6"></p>

6\. Напишите запрос на языке SQL, который возвращает все строки из таблицы `Recipe` с дополнительным столбцом, содержащем номер рецепта. Рецепты нумеруются целыми числами, начиная с 1, в __рамках каждого года__ в порядке их добавления в БД (столбец `submitted`). Получите результат в виде `pd.DataFrame`. Посчитайте и выведите на экран количество строк полученного `pd.DataFrame`, для которых сгенерированный номер кратен 50.

In [ ]:
request = pd.DataFrame(cur.execute('''SELECT *
                         FROM Recipe
                         '''))


In [ ]:
request[request[0]%50 == 0]

,0,1,2,3,4,5
38,44050,amish cake frosting icing,5,2002-10-25,yummmmmy! my amish mother-in-law shared this p...,NaN
136,250900,1930 s apple and nut pie,80,2007-09-04,i tore this recipe out of family circle. i ha...,NaN
138,245950,1960 tomato gravy,25,2007-08-12,from an index card in a recipe box i bought at...,9.0
220,228000,5 a day bulgur wheat salad,50,2007-05-14,found this on a diabetic website while surfing...,11.0
286,446600,a m couscous ww 5 pointsplus,10,2011-01-16,based on a recipe from the january/february 20...,4.0
...,...,...,...,...,...,...
29938,56250,zucchini herb bread,195,2003-03-12,this bread is very light and the moisture from...,10.0
29940,258100,zucchini ki bhaji stir fried zucchini,40,2007-10-10,"it's easy to cook and so tasty, i for one coul...",9.0
29957,377000,zucchini perini,30,2009-06-12,chef tom perini of the perini ranch steakhouse...,12.0
29990,71450,zucchini pineapple loaf cake,70,2003-09-15,i got this recipe from a local paper a long ti...,NaN


<p class="task" id="7"></p>

7\. Используя обобщенное табличное выражение и решение задачи 6, напишите запрос на языке SQL, который вернет количество строк, для которых сгенерированный номер кратен 50. Выполните запрос и выведите количество таких строк на экран.